In [31]:
import os, csv, json
from dotenv import load_dotenv
import time
from datetime import datetime
import schedule
import requests

import yfinance as yf
yf.pdr_override()

import pandas as pd
import pandas_ta as ta
import numpy as np

from google.oauth2 import service_account
import pygsheets

load_dotenv()

True

In [32]:
with open("..\service_account.json") as source:
    info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

CLIENT = pygsheets.authorize(service_account_file='..\service_account.json')
SHEET_URL = os.getenv('TRADE_SHEET')
data = CLIENT.sheet.get(os.getenv('TRADE_SHEET_ID'))
data['spreadsheetId']

'1WTJj8qkV9GeM_Fz8m2bLH3gZNNpSfRnqtzBiv3XaFsc'

In [45]:
def writePriceLog(title, new_rows):
    sheet = CLIENT.open_by_url(SHEET_URL)
    worksheet = sheet.worksheet_by_title(title)
    cells = worksheet.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')
    last_row = len(cells)
    
    worksheet = worksheet.insert_rows(last_row, number=1, values= new_rows)
    print(title + ' done...')

In [46]:
def initial_bars(pair, tf):
    ticker_yahoo = yf.Ticker(pair+'=x')
    bars = ticker_yahoo.history(period='3d', interval=tf)
    df = pd.DataFrame({
        'datetime': bars.index,
        'open': bars['Open'],
        'high': bars['High'],
        'low': bars['Low'],
        'close': bars['Close']
#         'volumn': bars['Volume']
    }) # convert alpha vantage data to use with strategy

    df['timestamp'] = pd.to_datetime(df['datetime']).values.astype(np.int64) // 10**9
    df['datetime'] = pd.to_datetime(df['timestamp'] * 10**9, utc=True).dt.tz_convert('Asia/Bangkok')
    df = df.astype({'datetime':'string'})
    df.reset_index(drop=True, inplace=True)

    return df

In [50]:
dt = initial_bars('EURUSD', '1h')
dt.head()

,datetime,open,high,low,close,timestamp
0,2022-08-08 06:00:00+07:00,1.017605,1.018019,1.017294,1.017501,1659913200
1,2022-08-08 07:00:00+07:00,1.017501,1.017501,1.016260,1.016880,1659916800
2,2022-08-08 08:00:00+07:00,1.017191,1.018019,1.016880,1.017812,1659920400
3,2022-08-08 09:00:00+07:00,1.018226,1.018745,1.017501,1.018330,1659924000
4,2022-08-08 10:00:00+07:00,1.018330,1.018952,1.018123,1.018952,1659927600


In [48]:
rows = dt.values.tolist()
writePriceLog('EURUSD_1h', rows)

done...
